#这个程序作用如下：
1.初步筛选了站点（根据时间文件个数，但并没有根据缺测情况来统计)。
2.1h降水转换为3h降水
3.最后输出df_final_2400.csv：为初步筛选后，3000多个站点的hourly数据，并且第3个小时的降水都已经转化为3h累积降水量。

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import datetime
import pandas as pd

data_path='/home/liuli/GTS_GFS_MOS/domestic_hourly_data/df_all.csv'
file0= pd.read_csv(data_path,sep=',')
df0=pd.DataFrame(file0)

In [ ]:
#print(df_all.info())
df_all=df0.drop_duplicates(subset = ['time','sta'],keep='last',inplace=False) #去掉重复的行
print(df_all)

In [3]:
#1.先把大df中某个站点的数据挑选出来
start='2018-01-01 00:00:00'
end_of_db = '2019-01-01 00:00:00'
date_index = pd.date_range(start, end_of_db, freq = '1h')
date_df = pd.DataFrame(date_index,columns=['time'])


#把df_all的时间转换成datatime
df_all['time']=pd.to_datetime(df_all['time'],format="%Y%m%d%H")
#df_all['time'] = df_all['time'].apply(lambda x : datetime.datetime.strptime(x,'%Y%m%d%H'))
#print(df_all['time'].dtype)

#print(type(df_all['time'].dt.hour[0]))
df_qc=df_all.drop_duplicates(subset = ['sta'],keep='last',inplace=False) #申明一个不重复站点的df
print(df_qc)

/home/liuli/anaconda3/envs/MOS/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


                        time       sta      ht    lat     lon       pr1  \
21226    2018-01-01 12:00:00     A8VV3  9999.0  13.30  109.90  999999.0   
55488    2018-01-02 12:00:00     DEDG2  9999.0   4.10   92.30  999999.0   
76655    2018-01-03 03:00:00      7XJX  9999.0  29.10  129.80  999999.0   
81182    2018-01-03 07:00:00    9V9130  9999.0  12.60  112.90  999999.0   
110863   2018-01-04 05:00:00      HONU  9999.0  36.80  143.90  999999.0   
149205   2018-01-05 06:00:00     DGQD2  9999.0   5.90   89.40  999999.0   
174941   2018-01-06 01:00:00    9V5332  9999.0  37.10  143.60  999999.0   
209009   2018-01-07 00:00:00      3FOB  9999.0   5.30   91.50  999999.0   
242942   2018-01-08 00:00:00      CQYN  9999.0  50.60  178.30  999999.0   
272632   2018-01-08 22:00:00    9V3440  9999.0  54.40  176.40  999999.0   
281359   2018-01-09 05:00:00      WAZV  9999.0  46.20  151.30  999999.0   
281366   2018-01-09 05:00:00      WKPM  9999.0  25.40   80.00  999999.0   
319323   2018-01-10 07:00

In [ ]:
#df2=df_all[df_all['sta'].isin(['53603'])]
#print(df2)
#for i in df_qc.sta:  #站点不重复
#    #print(i)
#    #print(type(i))
#    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行 
#    df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
#    df2=df2.reset_index(drop=True)
#    if len(df2)>4000:
#        print(df2)

In [ ]:
############开始循环读取每个站点的数据，并判断此站点的有效数据量
df_final=pd.DataFrame(columns=['time', 'sta', 'ht', 'lat', 'lon', 'pr1', 'pr24', 'pr6', 'ps', 'psl', 't2m', 'td', 'wdir', 'wspd'])

for i in df_qc.sta:  #站点不重复
    
    #print(type(i))
    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行
    if len(df2)>=2400: ####按照3小时一次，每天8次，一年8*365*80%（有效数据）
        print(i)
        df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
        df2=df2.reset_index(drop=True)
        #print(df2)         
        # merge日期补充完整数据
        df2 = pd.merge(date_df,df2, on=['time'], how='left')
        df2['sta'].fillna(i,inplace=True)
        #df2['sta']=df2['sta'].replace('NaN',i)
        #判断03时，处理为3小时累计降水
        for index, item in enumerate(df2['time'].values):
            #从第三行开始，把1h降水量累加成为3h降水量
            if index>=2 and df2['time'].dt.hour[index]%3==0 and df2['pr1'].values[index]<1000.0:
#            if index>=2 and df2['time'].dt.hour[index]%3==0:                
#                print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
                if df2['pr1'].values[index-1]<1000.0 and df2['pr1'].values[index-2]<1000.0:            
                    df2['pr1'].values[index]=df2['pr1'].values[index]+df2['pr1'].values[index-1]+df2['pr1'].values[index-2]
                    print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
                    
                else:
                    df2['pr1'].values[index]=999999.0
                    print(index,item,df2['sta'].values[index],df2['pr1'].values[index])
        df_final=pd.concat([df_final,df2],ignore_index=True)            
    #print(df2)
    #print(df2.info())
    del df2

#这种根据某一列数值，赋值另一列适合比较简单的场景：print(df_all.loc[(float(df_all.time[-2:])%3 ==0), 'pr1'] )
df_final

58806
3 2018-01-01T03:00:00.000000000 58806 999999.0
6 2018-01-01T06:00:00.000000000 58806 999999.0
9 2018-01-01T09:00:00.000000000 58806 999999.0
12 2018-01-01T12:00:00.000000000 58806 999999.0
15 2018-01-01T15:00:00.000000000 58806 999999.0
18 2018-01-01T18:00:00.000000000 58806 999999.0
21 2018-01-01T21:00:00.000000000 58806 999999.0
24 2018-01-02T00:00:00.000000000 58806 999999.0
27 2018-01-02T03:00:00.000000000 58806 999999.0
30 2018-01-02T06:00:00.000000000 58806 999999.0
33 2018-01-02T09:00:00.000000000 58806 999999.0
36 2018-01-02T12:00:00.000000000 58806 999999.0
39 2018-01-02T15:00:00.000000000 58806 999999.0
42 2018-01-02T18:00:00.000000000 58806 999999.0
45 2018-01-02T21:00:00.000000000 58806 999999.0
48 2018-01-03T00:00:00.000000000 58806 999999.0
51 2018-01-03T03:00:00.000000000 58806 999999.0
54 2018-01-03T06:00:00.000000000 58806 999999.0
57 2018-01-03T09:00:00.000000000 58806 999999.0
60 2018-01-03T12:00:00.000000000 58806 999999.0
63 2018-01-03T15:00:00.000000000 5880

699 2018-01-30T03:00:00.000000000 58806 999999.0
702 2018-01-30T06:00:00.000000000 58806 999999.0
705 2018-01-30T09:00:00.000000000 58806 999999.0
708 2018-01-30T12:00:00.000000000 58806 999999.0
711 2018-01-30T15:00:00.000000000 58806 999999.0
714 2018-01-30T18:00:00.000000000 58806 999999.0
717 2018-01-30T21:00:00.000000000 58806 999999.0
720 2018-01-31T00:00:00.000000000 58806 999999.0
723 2018-01-31T03:00:00.000000000 58806 999999.0
726 2018-01-31T06:00:00.000000000 58806 999999.0
729 2018-01-31T09:00:00.000000000 58806 999999.0
732 2018-01-31T12:00:00.000000000 58806 999999.0
735 2018-01-31T15:00:00.000000000 58806 999999.0
738 2018-01-31T18:00:00.000000000 58806 999999.0
741 2018-01-31T21:00:00.000000000 58806 999999.0
744 2018-02-01T00:00:00.000000000 58806 999999.0
747 2018-02-01T03:00:00.000000000 58806 999999.0
750 2018-02-01T06:00:00.000000000 58806 999999.0
753 2018-02-01T09:00:00.000000000 58806 999999.0
756 2018-02-01T12:00:00.000000000 58806 999999.0
759 2018-02-01T15:00

1353 2018-02-26T09:00:00.000000000 58806 999999.0
1356 2018-02-26T12:00:00.000000000 58806 999999.0
1359 2018-02-26T15:00:00.000000000 58806 999999.0
1362 2018-02-26T18:00:00.000000000 58806 999999.0
1365 2018-02-26T21:00:00.000000000 58806 999999.0
1368 2018-02-27T00:00:00.000000000 58806 999999.0
1371 2018-02-27T03:00:00.000000000 58806 999999.0
1374 2018-02-27T06:00:00.000000000 58806 999999.0
1377 2018-02-27T09:00:00.000000000 58806 999999.0
1380 2018-02-27T12:00:00.000000000 58806 999999.0
1383 2018-02-27T15:00:00.000000000 58806 999999.0
1386 2018-02-27T18:00:00.000000000 58806 999999.0
1389 2018-02-27T21:00:00.000000000 58806 999999.0
1392 2018-02-28T00:00:00.000000000 58806 999999.0
1395 2018-02-28T03:00:00.000000000 58806 999999.0
1398 2018-02-28T06:00:00.000000000 58806 999999.0
1401 2018-02-28T09:00:00.000000000 58806 999999.0
1404 2018-02-28T12:00:00.000000000 58806 999999.0
1407 2018-02-28T15:00:00.000000000 58806 999999.0
1410 2018-02-28T18:00:00.000000000 58806 999999.0


2031 2018-03-26T15:00:00.000000000 58806 999999.0
2034 2018-03-26T18:00:00.000000000 58806 999999.0
2037 2018-03-26T21:00:00.000000000 58806 999999.0
2040 2018-03-27T00:00:00.000000000 58806 999999.0
2043 2018-03-27T03:00:00.000000000 58806 999999.0
2046 2018-03-27T06:00:00.000000000 58806 999999.0
2049 2018-03-27T09:00:00.000000000 58806 999999.0
2052 2018-03-27T12:00:00.000000000 58806 999999.0
2055 2018-03-27T15:00:00.000000000 58806 999999.0
2058 2018-03-27T18:00:00.000000000 58806 999999.0
2061 2018-03-27T21:00:00.000000000 58806 999999.0
2064 2018-03-28T00:00:00.000000000 58806 999999.0
2067 2018-03-28T03:00:00.000000000 58806 999999.0
2070 2018-03-28T06:00:00.000000000 58806 999999.0
2073 2018-03-28T09:00:00.000000000 58806 999999.0
2076 2018-03-28T12:00:00.000000000 58806 999999.0
2079 2018-03-28T15:00:00.000000000 58806 999999.0
2082 2018-03-28T18:00:00.000000000 58806 999999.0
2085 2018-03-28T21:00:00.000000000 58806 999999.0
2088 2018-03-29T00:00:00.000000000 58806 999999.0


2709 2018-04-23T21:00:00.000000000 58806 999999.0
2712 2018-04-24T00:00:00.000000000 58806 999999.0
2715 2018-04-24T03:00:00.000000000 58806 999999.0
2718 2018-04-24T06:00:00.000000000 58806 999999.0
2721 2018-04-24T09:00:00.000000000 58806 999999.0
2724 2018-04-24T12:00:00.000000000 58806 999999.0
2727 2018-04-24T15:00:00.000000000 58806 999999.0
2730 2018-04-24T18:00:00.000000000 58806 999999.0
2733 2018-04-24T21:00:00.000000000 58806 999999.0
2736 2018-04-25T00:00:00.000000000 58806 999999.0
2739 2018-04-25T03:00:00.000000000 58806 999999.0
2742 2018-04-25T06:00:00.000000000 58806 999999.0
2745 2018-04-25T09:00:00.000000000 58806 999999.0
2748 2018-04-25T12:00:00.000000000 58806 999999.0
2751 2018-04-25T15:00:00.000000000 58806 999999.0
2754 2018-04-25T18:00:00.000000000 58806 999999.0
2757 2018-04-25T21:00:00.000000000 58806 999999.0
2760 2018-04-26T00:00:00.000000000 58806 999999.0
2763 2018-04-26T03:00:00.000000000 58806 999999.0
2766 2018-04-26T06:00:00.000000000 58806 999999.0


...

7800 2018-11-22T00:00:00.000000000 58823 999999.0
7803 2018-11-22T03:00:00.000000000 58823 999999.0
7806 2018-11-22T06:00:00.000000000 58823 999999.0
7809 2018-11-22T09:00:00.000000000 58823 999999.0
7812 2018-11-22T12:00:00.000000000 58823 999999.0
7815 2018-11-22T15:00:00.000000000 58823 999999.0
7818 2018-11-22T18:00:00.000000000 58823 999999.0
7821 2018-11-22T21:00:00.000000000 58823 999999.0
7824 2018-11-23T00:00:00.000000000 58823 999999.0
7827 2018-11-23T03:00:00.000000000 58823 999999.0
7830 2018-11-23T06:00:00.000000000 58823 999999.0
7833 2018-11-23T09:00:00.000000000 58823 999999.0
7836 2018-11-23T12:00:00.000000000 58823 999999.0
7839 2018-11-23T15:00:00.000000000 58823 999999.0
7842 2018-11-23T18:00:00.000000000 58823 999999.0
7845 2018-11-23T21:00:00.000000000 58823 999999.0
7848 2018-11-24T00:00:00.000000000 58823 999999.0
7851 2018-11-24T03:00:00.000000000 58823 999999.0
7854 2018-11-24T06:00:00.000000000 58823 999999.0
7857 2018-11-24T09:00:00.000000000 58823 999999.0


8505 2018-12-21T09:00:00.000000000 58823 999999.0
8508 2018-12-21T12:00:00.000000000 58823 999999.0
8511 2018-12-21T15:00:00.000000000 58823 999999.0
8514 2018-12-21T18:00:00.000000000 58823 999999.0
8517 2018-12-21T21:00:00.000000000 58823 999999.0
8520 2018-12-22T00:00:00.000000000 58823 999999.0
8523 2018-12-22T03:00:00.000000000 58823 999999.0
8526 2018-12-22T06:00:00.000000000 58823 999999.0
8529 2018-12-22T09:00:00.000000000 58823 999999.0
8532 2018-12-22T12:00:00.000000000 58823 999999.0
8535 2018-12-22T15:00:00.000000000 58823 999999.0
8538 2018-12-22T18:00:00.000000000 58823 999999.0
8541 2018-12-22T21:00:00.000000000 58823 999999.0
8544 2018-12-23T00:00:00.000000000 58823 999999.0
8547 2018-12-23T03:00:00.000000000 58823 999999.0
8550 2018-12-23T06:00:00.000000000 58823 999999.0
8553 2018-12-23T09:00:00.000000000 58823 999999.0
8556 2018-12-23T12:00:00.000000000 58823 999999.0
8559 2018-12-23T15:00:00.000000000 58823 999999.0
8562 2018-12-23T18:00:00.000000000 58823 999999.0


468 2018-01-20T12:00:00.000000000 58824 999999.0
471 2018-01-20T15:00:00.000000000 58824 999999.0
474 2018-01-20T18:00:00.000000000 58824 999999.0
477 2018-01-20T21:00:00.000000000 58824 999999.0
480 2018-01-21T00:00:00.000000000 58824 999999.0
483 2018-01-21T03:00:00.000000000 58824 999999.0
486 2018-01-21T06:00:00.000000000 58824 999999.0
489 2018-01-21T09:00:00.000000000 58824 999999.0
492 2018-01-21T12:00:00.000000000 58824 999999.0
495 2018-01-21T15:00:00.000000000 58824 999999.0
498 2018-01-21T18:00:00.000000000 58824 999999.0
501 2018-01-21T21:00:00.000000000 58824 999999.0
504 2018-01-22T00:00:00.000000000 58824 999999.0
507 2018-01-22T03:00:00.000000000 58824 999999.0
510 2018-01-22T06:00:00.000000000 58824 999999.0
513 2018-01-22T09:00:00.000000000 58824 999999.0
516 2018-01-22T12:00:00.000000000 58824 999999.0
519 2018-01-22T15:00:00.000000000 58824 999999.0
522 2018-01-22T18:00:00.000000000 58824 999999.0
525 2018-01-22T21:00:00.000000000 58824 999999.0
528 2018-01-23T00:00

1182 2018-02-19T06:00:00.000000000 58824 999999.0
1185 2018-02-19T09:00:00.000000000 58824 999999.0
1188 2018-02-19T12:00:00.000000000 58824 999999.0
1191 2018-02-19T15:00:00.000000000 58824 999999.0
1194 2018-02-19T18:00:00.000000000 58824 999999.0
1197 2018-02-19T21:00:00.000000000 58824 999999.0
1200 2018-02-20T00:00:00.000000000 58824 999999.0
1203 2018-02-20T03:00:00.000000000 58824 999999.0
1206 2018-02-20T06:00:00.000000000 58824 999999.0
1209 2018-02-20T09:00:00.000000000 58824 999999.0
1212 2018-02-20T12:00:00.000000000 58824 999999.0
1215 2018-02-20T15:00:00.000000000 58824 999999.0
1218 2018-02-20T18:00:00.000000000 58824 999999.0
1221 2018-02-20T21:00:00.000000000 58824 999999.0
1224 2018-02-21T00:00:00.000000000 58824 999999.0
1227 2018-02-21T03:00:00.000000000 58824 999999.0
1230 2018-02-21T06:00:00.000000000 58824 999999.0
1233 2018-02-21T09:00:00.000000000 58824 999999.0
1239 2018-02-21T15:00:00.000000000 58824 999999.0
1242 2018-02-21T18:00:00.000000000 58824 999999.0


1884 2018-03-20T12:00:00.000000000 58824 999999.0
1887 2018-03-20T15:00:00.000000000 58824 999999.0
1890 2018-03-20T18:00:00.000000000 58824 999999.0
1893 2018-03-20T21:00:00.000000000 58824 999999.0
1896 2018-03-21T00:00:00.000000000 58824 999999.0
1899 2018-03-21T03:00:00.000000000 58824 999999.0
1902 2018-03-21T06:00:00.000000000 58824 999999.0
1905 2018-03-21T09:00:00.000000000 58824 999999.0
1908 2018-03-21T12:00:00.000000000 58824 999999.0
1911 2018-03-21T15:00:00.000000000 58824 999999.0
1914 2018-03-21T18:00:00.000000000 58824 999999.0
1917 2018-03-21T21:00:00.000000000 58824 999999.0
1920 2018-03-22T00:00:00.000000000 58824 999999.0
1923 2018-03-22T03:00:00.000000000 58824 999999.0
1926 2018-03-22T06:00:00.000000000 58824 999999.0
1929 2018-03-22T09:00:00.000000000 58824 999999.0
1932 2018-03-22T12:00:00.000000000 58824 999999.0
1935 2018-03-22T15:00:00.000000000 58824 999999.0
1938 2018-03-22T18:00:00.000000000 58824 999999.0
1941 2018-03-22T21:00:00.000000000 58824 999999.0


In [ ]:
df_final.to_csv('./df_final_2400.csv', sep=',', header=True, index=False)